Pseudocode for Permutation Core Butson-Type Complex Hadamard Matrices:

* Select dimension $n$ for matrix
* Generate multiples of $\Phi_n(x)$ with sum of coefficient magnitudes equal to $n$
* Normalize so that they're all positive coefficients by multiplying by $x^{n/2}$ for the negative ones

Definition: A cycle is a full sum of roots of unity, possibly rotated by a scalar. Geometrically, it represents a regular polygon on the unit circle.

Let $l=p_1^{a_1}\cdots p_k^{a_k}$, and assume that $\lambda_i\in\mathbb{Z}_l$ satisfy $\lambda_1+\cdots+\lambda_N=0$.

If $k\leq 2$, then $\sum p_i^{a_i}$ is a sum of cycles with coefficients in $\mathbb{N}$.

If $k\geq 3$, then $\sum p_i^{a_i}$ is a sum of cycles with coefficients in $\mathbb{Z}$, but not necessarily in $\mathbb{n}$.

Here, we'll focus our search on products of 2 prime powers, i.e. 6. That way, we'll have sums of cycles.

The gist of what I want to write here is going to be the following:

Function for generating polynomials that sum to 0 in our root of unity.
* For roots with 1 prime divisor, this is a fairly trivial exercise because it usually doesn't work.
* For roots with 2 prime divisors, they can be broken down into rotated full sums of prime roots of unity.
* For roots with 3 or more prime divisors, they can be broken down into POSITIVE OR NEGATIVE full sums of prime roots of unity.

In the meantime, focus on writing the program for the roots with 2 prime divisors. It's mostly a partition problem for that one.

Once I've got the polynomials from the first function, I then want to run them through multiple ways of generating Butson-type CHMs.
* For the permutation core Butson-type CHMs, I just want to see if I can take said rows and use an individual row to generate an entire matrix.
* For the general Butson-type CHMs, I'd need to use a more elaborate function that takes multiple candidate rows (along with their permutations) and determines if they work, before moving on to other candidates. Not sure how to handle the indexing and sorting for this one in particular.
* It probably *does* make sense to try to write things with a similar workflow to the way the AI wrote it, i.e. dedicated functions for generating acceptable rows, permuting said rows (might be a library thing), testing orthogonality between a candidate row and a list of other (fixed-position) rows, and constructing a matrix.
* The main thing that seems to pose a challenge is going to be the iteration process. I'm not sure how to go about this, but I'm sure there's a way.

# AS OF 10/8/2025 (Tibor)

Currently, this program just generates a list of vanishing sums of $d$th roots of unity with $L_1$ norm equal to $n$, where $e^{2\pi i/d}$ is the root of unity in question and $n$ is the dimension of the matrix. So far, I've only accounted for values of $d$ with at most 2 prime factors.

It now generates all permutation-core Butson-type CHMs with a given root of unity and dimension, so long as $d$ has at most 2 prime factors.

I haven't gotten around to writing the function to generate ALL Butson-type CHMs of these dimensions, but the logic is essentially the same.

In [39]:
#type: ignore

import itertools
import multiprocessing as mp

def cycle_exponents(k, d, shift=0):
	"""
	Generate a cycle of length k of dth roots of unity, where k is a prime divisor of d, rotated by shift.
	"""
	step = d // k
	return [(shift + j * step) % d for j in range(k)]


def partitions_with_parts(n, parts):
	"""
	Generate all integer combinations of given parts that sum to n.

	Args:
	- n (int): the total to reach (L1 norm)
	- parts (list[int]): allowed part sizes, e.g. [p, q]

	Returns:
	- list of tuples (a1, a2, ..., ak) where sum(ai * parts[i]) == n
	"""
	results = []
	def helper(idx, remaining, coeffs):
		"""
		Generate the actual integer combinations of parts recursively. Currently only valid for up to 2 primes.
		
		Args:
		- idx (int): the index of which part is actually being incremented
		- remaining (int): the remainder; it tells us when we've got a valid combination or have missed our window to find one
		- coeffs (list[tuple[int]]): coefficients of the parts from the parent function
		"""
		if idx == len(parts) - 1:
			if remaining % parts[idx] == 0:
				results.append(tuple(coeffs + [remaining // parts[idx]]))
			return
		for count in range(remaining // parts[idx] + 1):
			helper(idx + 1, remaining - count * parts[idx], coeffs + [count])
	helper(0, n, [])
	return results


def generate_L1_vectors(n, d, require_zero=True):
	"""
	Generate a list of vectors of dth roots of unity which have L1 norm n.

	Args:
	- n (int): The L1 norm of the vectors
	- d (int): The primitive root of unity out of which these are constructed
	- require_zero (boolean): If true, forces the vectors to contain at least one 1 (exponent 0) to account for dephased matrix

	Returns:
	- List of all vectors satisfying the above conditions
	"""

	# Make sure n and d are integers
	if not isinstance(n, (int, Integer)):
		raise TypeError(f"n must be an integer, not {type(n).__name__}")
	if not isinstance(d, (int, Integer)):
		raise TypeError(f"d must be an integer, not {type(d).__name__}")

	# Value checks
	if n < 0:
		raise ValueError(f"n must be nonnegative (L1 norm), got {n}")
	if d < 2:
		raise ValueError(f"d must be a natural number ≥ 2, got {d}")

	primes = prime_divisors(d)

	if len(primes) > 2:
		raise ValueError(f"d must have at most 2 prime factors (at least for now), got {len(primes)}")

	partitions = partitions_with_parts(n, primes)
	all_vectors = set()

	for coeffs in partitions:
		prime_counts = list(zip(primes, coeffs))
		# Build all combinations of shifts for all cycles
		shift_space = []
		for k, count in prime_counts:
			# each cycle of size k can be rotated independently
			shift_space.append(product(range(d // k), repeat=count))
		# Cartesian product over all primes’ shift sets
		for combo in product(*shift_space):
			vector = []
			for (k, count), shifts in zip(prime_counts, combo):
				for shift in shifts:
					vector.extend(cycle_exponents(k, d, shift))
			vector.sort()
			if require_zero and 0 not in vector:
				continue
			all_vectors.add(tuple(vector))

	return [list(v) for v in sorted(all_vectors)]

def rows_orthogonal(u, v, d):
	"""
	Determine whether or not a vector u is orthogonal to a vector v,
	where entries are given as exponents of a dth root of unity.
	"""
	R = CyclotomicField(d)
	zeta = R.gen()
	if len(u) != len(v):
	    raise ValueError(f"Vectors {i} and {j} differ in length")
	inner = sum(zeta**(u_i - v_i) for u_i, v_i in zip(u, v))
	return inner == 0

def row_orthogonal_to_list(candidate_row, existing_rows, d):
	"""
	Check whether 'candidate_row' is orthogonal to every vector
	in 'existing_rows' under dth roots of unity.
	"""
	return all(rows_orthogonal_exact(candidate_row, row, d) for row in existing_rows)

def valid_permutation(candidate_row, existing_rows):
	"""
	Return True if candidate_row respects the "don't swap identical positions in previous rows" rule.
	"""
	n = len(candidate_row)
	for i in range(n):
		for j in range(i+1, n):
			if candidate_row[i] > candidate_row[j]:
				# Check if positions i and j are identical in all previous rows
				if all(row[i] == row[j] for row in existing_rows):
					return False
	return True


def build_permutation_core_CHMs(core_coeffs, n, d):
	"""
	core_coeffs: list of integers
	n: size of the CHM
	d: dth root of unity, i.e. e^{2pi i/d}
	Returns: list of matrices (each matrix is a list of rows, each row is a list)
	"""
	first_row = [0]*n
	second_row = core_coeffs
	fixed_first = core_coeffs[0]
	tail = core_coeffs[1:]
	results = []

	def backtrack(existing_rows):
		if len(existing_rows) == n:
	    	# Deep copy of matrix
			results.append([row[:] for row in existing_rows])
			return

    	# Generate only unique permutations of tail
		for perm in Permutations(tail):
			candidate_row = [fixed_first] + list(perm)

			# Canonical ordering: only consider rows >= last row
			if candidate_row > existing_rows[-1] and \
				all(rows_orthogonal(candidate_row, row, d) for row in existing_rows) and \
					valid_permutation(candidate_row, existing_rows):
				existing_rows.append(candidate_row)
				backtrack(existing_rows)
				existing_rows.pop()

	backtrack([first_row, second_row])
	return results

def build_Butson_CHMs(candidate_rows, n, d):
	"""
	Generate all n x n Butson-type CHMs whose rows are drawn (as permutations)
	from the given list of candidate rows.

	Args:
	- candidate_rows: list of lists (rows generated by generate_L1_vectors)
	- n: matrix size
	- d: dth root of unity, i.e. e^{2pi i/d}

	Returns:
	- list of CHMs (each CHM is a list of n rows, each row a list of ints)
	"""
	results = []
	first_row = [0]*n	# standard dephased first row

	def backtrack(existing_rows):
		if len(existing_rows) == n:
			results.append([row[:] for row in existing_rows])
			return

		for base_row in candidate_rows:
			fixed_first = base_row[0]
			tail = base_row[1:]
			for perm in Permutations(tail):
				row = [fixed_first] + list(perm)

				# skip duplicate row
				if row in existing_rows:
					continue

				# ensure orthogonality with all previous rows
				if not all(rows_orthogonal(row, prev, d) for prev in existing_rows):
					continue

				# symmetry breaking: don't swap identical columns of previous rows
				if not valid_permutation(row, existing_rows):
					continue

				# canonical ordering (optional, avoids reordering symmetry)
				if row < existing_rows[-1]:
					continue

				existing_rows.append(row)
				backtrack(existing_rows)
				existing_rows.pop()

	# start recursion with dephased first row
	backtrack([first_row])
	return results

def write_chms_to_file(chms, filename="output.txt"):
	with open(f"output/{filename}", "w") as f:
		if chms:
			for i, mat in enumerate(chms, 1):
				f.write(f"Matrix {i}:\n")
				for row in mat:
					f.write(" ".join(map(str, row)) + "\n")
				f.write("\n")
		else:
			f.write(f"No matrices found.")

def run_parallel(n, d):
	vector_list = generate_L1_vectors(n, d)

	# Create a multiprocessing pool with as many workers as CPU cores
	with mp.Pool(mp.cpu_count()) as pool:
		# Compute permutation CHMs in parallel
		permutation_results = pool.starmap(
			build_permutation_core_CHMs,
			[(list(coeffs), n, d) for coeffs in vector_list]
		)

	# Flatten results (since each call returns a list of matrices)
	permutation_chms = [chm for sublist in permutation_results for chm in sublist]
	write_chms_to_file(permutation_chms, f"Permutation-core H({d}, {n}).txt")

	# Now do the Butson CHMs (also parallelized if that function is independent)
	with mp.Pool(mp.cpu_count()) as pool:
		butson_results = pool.starmap(
			build_Butson_CHMs,
			[(vector_list, n, d)]
		)

	# Flatten results again
	butson_chms = [chm for sublist in butson_results for chm in sublist]
	write_chms_to_file(butson_chms, f"Butson-type H({d}, {n}).txt")


if __name__ == "__main__":
	d = 6  # e^{2πi/d}
	run_parallel(8, d)

Process ForkPoolWorker-685:
Process ForkPoolWorker-682:
Process ForkPoolWorker-684:
Process ForkPoolWorker-681:
Process ForkPoolWorker-680:
Process ForkPoolWorker-676:
Process ForkPoolWorker-691:
Process ForkPoolWorker-699:
Process ForkPoolWorker-690:
Process ForkPoolWorker-673:
Process ForkPoolWorker-700:
Process ForkPoolWorker-695:
Process ForkPoolWorker-689:
Process ForkPoolWorker-675:
Process ForkPoolWorker-696:
Process ForkPoolWorker-678:
Process ForkPoolWorker-702:
Process ForkPoolWorker-679:
Process ForkPoolWorker-698:
Process ForkPoolWorker-697:
Process ForkPoolWorker-677:
Process ForkPoolWorker-704:
Process ForkPoolWorker-693:
Process ForkPoolWorker-703:
Process ForkPoolWorker-683:
Process ForkPoolWorker-701:
Process ForkPoolWorker-694:
Process ForkPoolWorker-692:
Process ForkPoolWorker-686:
Process ForkPoolWorker-688:
Traceback (most recent call last):
Process ForkPoolWorker-674:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call

KeyboardInterrupt: 